# Using EDHEC Risk Kit Part 2
### Explore use of the library to:
- calculate monthly parametric guassian VaR
- calculate monthly parametric gaussian VaR with Cornish-Fisher Adjustment
- retrieve weights in a portfolio
- build MSR and GMV portfolios
- Calculate annual volatility of a portfolio containing multiple assets

In [1]:
import numpy as np
import pandas as pd
import edhec_risk_kit as erk
import matplotlib.pyplot as plt

### Load the hedge fund index returns
- Considers years from 1980 to 2015

In [2]:
hfi = erk.get_hfi_returns()['1980':'2015']

### What was the Monthly Parametric Gaussian VaR at the 1% level of the Emerging Markets strategy?

In [3]:
erk.var_gaussian(hfi['Emerging Markets'], level=1)

0.0727702691677074

### What was the Monthly Parametric Gaussian VaR at the 1% level of the Emerging Markets strategy with the Cornish-Fisher Adjustment?

In [4]:
erk.var_gaussian(hfi['Emerging Markets'], level=1, modified=True)

0.13051127699820947

### What was the Monthly Historic VaR at the 1% level of the Emerging Markets strategy?

In [5]:
erk.var_historic(hfi['Emerging Markets'], level=1)

0.093961

### Load industry returns from 1980 to 2015
- Consider the Finance, Utility, Telecom, and Health industries

In [6]:
cols = ['Fin', 'Util', 'Telcm', 'Hlth']
ind = erk.get_ind_returns().loc['1980':'2015', cols]

### Calculate the weights for the maximum Sharpe ratio portfolio

In [7]:
er = erk.annualize_rets(ind, periods_per_year=12)
cov = ind.cov()
w_msr = erk.msr(riskfree_rate=0.02, er=er, cov=cov)
pd.DataFrame(w_msr, index=cols, columns=['Shape Ratio Weight'])

,Shape Ratio Weight
Fin,0.000000
Util,0.454315
Telcm,0.061605
Hlth,0.484080


### Calculate the weights for the global minimum variance portfolio

In [8]:
w_gmv = erk.gmv(cov=cov)
pd.DataFrame(w_gmv, index=cols, columns=['GMV Weight'])

,GMV Weight
Fin,0.000000
Util,0.581040
Telcm,0.169712
Hlth,0.249248


### Compare the annualized volatility of the MSR and GMV portfolios (weights calculated from data from 1980 to 2015) from 2016 to 2018

In [9]:
ind = erk.get_ind_returns().loc[:, cols]

er = erk.annualize_rets(
    ind['1980':'2015'], 
    periods_per_year=12
)

cov = ind['1980':'2015'].cov()
w_msr = erk.msr(riskfree_rate=0.02, er=er, cov=cov)
w_gmv = erk.gmv(cov=cov)

vol_msr = np.sqrt(12)*erk.portfolio_vol(
    weights=w_msr, 
    covmat=ind['2016':'2018'].cov()
)
vol_gmv = np.sqrt(12)*erk.portfolio_vol(
    weights=w_gmv, 
    covmat=ind['2016':'2018'].cov()
)

print('MSR 2016-2018 variance:', vol_msr)
print('GMV 2016-2018 variance:', vol_gmv)

MSR 2016-2018 variance: 0.0968449984016509
GMV 2016-2018 variance: 0.08990557036497361


### Calculate the annualized returns of the these two portfolios from 2016 to 2018

In [10]:
r = erk.annualize_rets(
    ind['2016':'2018'], 
    periods_per_year=12
)

r_msr = erk.portfolio_return(
    weights = w_msr, 
    returns = r
)

r_gmv = erk.portfolio_return(
    weights = w_gmv,
    returns = r
)

print('MSR 2016-2018 return:', r_msr)
print('GMV 2016-2018 return:', r_gmv)

MSR 2016-2018 return: 0.08369298270843524
GMV 2016-2018 return: 0.08893150684183913
